In [ ]:
import time
import csv
import threading
from robomaster import robot, blaster

# === Target ===
YAW_TARGETS = [-30, 0, 30]
PITCH_TARGET = 0

# === PID Parameters ===
Kp_pitch = 0.8
Ki_pitch = 0.01
Kd_pitch = 0.05

Kp_yaw = 1.8
Ki_yaw = 0.02
Kd_yaw = 0.05

TOLERANCE = 1.0
MAX_SPEED = 500
MIN_SPEED = -500

# === Global Variables ===
current_pitch = 0.0
current_yaw = 0.0
log_running = True

def clamp(value, min_val, max_val):
    return max(min_val, min(max_val, value))

def sub_angle_handler(angle_info):
    global current_pitch, current_yaw
    current_pitch = angle_info[0]
    current_yaw = angle_info[1]

def pid_loop(gimbal, target_pitch, target_yaw):
    global current_pitch, current_yaw

    prev_error_pitch = 0
    prev_error_yaw = 0
    integral_pitch = 0
    integral_yaw = 0

    while True:
        error_pitch = target_pitch - current_pitch
        error_yaw = target_yaw - current_yaw

        if abs(error_pitch) < TOLERANCE and abs(error_yaw) < TOLERANCE:
            break

        integral_pitch += error_pitch
        integral_yaw += error_yaw

        derivative_pitch = error_pitch - prev_error_pitch
        derivative_yaw = error_yaw - prev_error_yaw

        speed_pitch = (Kp_pitch * error_pitch) + (Ki_pitch * integral_pitch) + (Kd_pitch * derivative_pitch)
        speed_yaw = (Kp_yaw * error_yaw) + (Ki_yaw * integral_yaw) + (Kd_yaw * derivative_yaw)

        speed_pitch = clamp(speed_pitch, MIN_SPEED, MAX_SPEED)
        speed_yaw = clamp(speed_yaw, MIN_SPEED, MAX_SPEED)

        gimbal.drive_speed(pitch_speed=speed_pitch, yaw_speed=speed_yaw)

        prev_error_pitch = error_pitch
        prev_error_yaw = error_yaw

        time.sleep(0.05)  # 20Hz

def log_angle_continuous(writer, start_time):
    """Log pitch/yaw in real-time"""
    global log_running
    while log_running:
        timestamp = round(time.time() - start_time, 3)
        writer.writerow([timestamp, round(current_pitch, 2), round(current_yaw, 2)])
        time.sleep(0.05)  # log ทุก 0.05 วินาที

if __name__ == '__main__':
    ep_robot = robot.Robot()
    ep_robot.initialize(conn_type="ap")

    gimbal = ep_robot.gimbal
    gun = ep_robot.blaster

    # Subscribe เพื่อดึงค่ามุม
    gimbal.sub_angle(freq=10, callback=sub_angle_handler)

    with open("gimbal_time_response.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["time", "pitch", "yaw"])

        sequence = [YAW_TARGETS[0], YAW_TARGETS[1], YAW_TARGETS[2], YAW_TARGETS[0]]

        start_time = time.time()

        # เริ่ม thread บันทึกค่า yaw/pitch
        log_running = True
        log_thread = threading.Thread(target=log_angle_continuous, args=(writer, start_time))
        log_thread.start()

        for yaw in sequence:
            pid_loop(gimbal, target_pitch=PITCH_TARGET, target_yaw=yaw)
            gun.fire(fire_type=blaster.INFRARED_FIRE)
            time.sleep(0.5)

        # หยุดการ log และรอ thread จบ
        log_running = False
        log_thread.join()

    gimbal.unsub_angle()
    gimbal.recenter(pitch_speed=100, yaw_speed=100).wait_for_completed()
    ep_robot.close()
